In [2]:
import pandas as pd
import sqlite3


In [3]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [13]:
import json
import pandas as pd

with open("users.json", "r", encoding="utf-8") as f:
    content = f.read()

# remove starting [ and ending ]
content = content.strip().lstrip("[").rstrip("]")

# split objects safely
objects = content.split("},")
objects = [obj + "}" if not obj.endswith("}") else obj for obj in objects]

rows = [json.loads(obj) for obj in objects]

users = pd.DataFrame(rows)
users.head()


JSONDecodeError: Extra data: line 8 column 1 (char 105)

In [ ]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as file:
    conn.executescript(file.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


In [ ]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)
orders_users.head()


In [ ]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)
final_df.head()


In [ ]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [1]:
import pandas as pd
print(pd.__version__)


3.0.0
